In [1]:
from dask.distributed import Client
import os

import dask

In [2]:
from dask_cuda import LocalCUDACluster
cluster = LocalCUDACluster()

In [3]:
c = Client(cluster)
c

Client Scheduler: tcp://127.0.0.1:39711 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 2 Cores: 2 Memory: 50.39 GB


In [4]:
from nccl_example import NCCL_Clique, unique_id
import random
from dask.distributed import wait

In [15]:
from dask import delayed
import dask.dataframe as dd
from helpers import _get_mg_info, gen_dask_cudf
import numba.cuda
import cudf
import numpy as np

In [16]:
class Dask_NCCL_Demo:
    
    def __init__(self, client):
        self.client = client
    
    @staticmethod
    def func_parse_host_port_(address):
        if '://' in address:
            address = address.rsplit('://', 1)[1]
        host, port = address.split(':')
        port = int(port)
        return host, port
    
    @staticmethod
    def func_init_(workerId, nWorkers, uniqueId):
        w = dask.distributed.get_worker()
        
        print("UNIQUEID: "  + str(uniqueId))
        
        print("Hello World! from ip=%s worker=%s/%d" % \
              (w.address, w.name, nWorkers))
        a = NCCL_Clique(workerId, nWorkers)
        a.create_clique(uniqueId)

        return a
    
    @staticmethod
    def func_get_size_(world, r):
        return world.get_clique_size()
    
    @staticmethod
    def func_get_rank_(world, r):
        return world.get_rank()
    
    @staticmethod
    def func_get_device_(world, r):
        return world.get_device()

    @staticmethod
    def func_test_all_reduce_(world, r):
        return world.test_all_reduce()
    
    
    @staticmethod
    def get_meta(df):
        return df.iloc[:0]
    
    @staticmethod
    def func_test_reduce_on_partition(world_df, root_rank, r):
        """
        This function is executed on the workers and performs the necessary
        data preparation, as well as calling the cython-wrapped C++ "algorithm"
        function(s), returning a cuDF with results, if necessary. 
        
        The client will construct a Dask cuDF out of the cuDFs returned from
        this function, if necessary. 
        """
        
        world, df = world_df

        # Build output array only for the root rank
        if world.get_rank() == root_rank:
            out_gpu_mat = numba.cuda.to_device(np.zeros((df.shape[0], df.shape[1]),
                                                        dtype=np.float32, order = "F"))
            out_df = cudf.DataFrame(index=cudf.dataframe.RangeIndex(0, df.shape[0]))
            
        else:
            out_gpu_mat = numba.cuda.device_array((1, 1), dtype=np.float32)
            out_df = None

        # Execute our cython-wrapped C++ "algorithm"
        world.test_on_partition(df, root_rank, out_gpu_mat)
                 
        # Build cudf with results
        if world.get_rank() == root_rank:
            for i in range(0, out_gpu_mat.shape[1]):
                out_df[str(i)] = out_gpu_mat[:, i]
        return out_df

    def test_end_to_end(self, dask_cudf):
        """
        An end to end example to mimic a typical Dask cuML algorithm using
        OPG semantics. This function is executed on the client.
        
        The following steps are taken with Dask cuDF as input:
        1. Co-locate Dask cuDF partitions with cuml NCCL clique object on each worker
        2. Run the algorithm, extracting the Numba device memory pointer for each partition
           and allocating necessary output memory on device for constructing the cuDF 
           partition(s) that will be returned to the user. 
        3. Construct Dask cuDF from the futures containing the cuDFs returned from local 
           "algorithm" functions on each worker. 
        """
        
        # Keep the futures around so the GPU memory doesn't get
        # deallocated on the workers.
        gpu_futures, cols = c.sync(_get_mg_info, X)
        
        # Combine dask_cudf partitions with their local "world" instance. 
        worker_world_map = dict(map(lambda x: (x[1], x[2]), self.clique))
        root_rank = demo.get_rank()[0]
        combined = list(map(delayed, [(worker_world_map[w], f) for w, f in gpu_futures]))
        combined = c.compute(combined)
        wait(combined)

        # Run our "algorithm" to perform reduce 
        f = [c.submit(Dask_NCCL_Demo.func_test_reduce_on_partition, f, root_rank, random.random()) for f in combined]
        wait(f)
        
        # Convert result back into a dask_cudf
        dfs = [d for d in f if d.type != type(None)]
        meta = c.submit(self.get_meta, dfs[0]).result()
        ddf = dd.from_delayed(dfs, meta=meta)
        
        return ddf
    
    def worker_ranks(self):
        return dict(list(map(lambda x: (x[1], x[0]), self.clique)))
    
    
    def run_func_on_workers(self, func):
        f = [c.submit(func, a, random.random()) for i, w, a in self.clique]
        wait(f)
        return [a.result() for a in f]

    def get_workers_(self):
        return list(map(lambda x: Dask_NCCL_Demo.func_parse_host_port_(x), self.client.has_what().keys()))
    
    def init(self, uniqueId):
        workers = self.get_workers_()
        workers_indices = list(zip(workers, range(len(workers))))

        self.clique = [(idx, worker, self.client.submit(Dask_NCCL_Demo.func_init_, 
                                           idx, 
                                           len(workers), 
                                           uniqueId,
                                           workers=[worker]))
             for worker, idx in workers_indices]
        
    def get_clique_size(self):
        return self.run_func_on_workers(Dask_NCCL_Demo.func_get_size_)

    def get_rank(self):
        return self.run_func_on_workers(Dask_NCCL_Demo.func_get_rank_)
    
    def test_all_reduce(self):
        return self.run_func_on_workers(Dask_NCCL_Demo.func_test_all_reduce_)

First demonstration will show that NCCL works within Dask as expected. This includes:
1. Creation of a NCCL clique using Dask workers to broadcast the ncclUniqueId
2. Demonstrating the NCCL clique successfully performs collective comms
3. All calls to the underlying NCCL comm are made through the cuML comms facade. 

In [17]:
world_id = unique_id()

In [18]:
demo = Dask_NCCL_Demo(c)
demo.init(world_id)

In [19]:
demo.get_clique_size()

[2, 2]

In [20]:
demo.get_rank()

[0, 1]

In [21]:
demo.worker_ranks()

{('127.0.0.1', 39646): 0, ('127.0.0.1', 43981): 1}

In [22]:
for result in demo.test_all_reduce():
    assert result == True

In [23]:
demo.get_workers_()

[('127.0.0.1', 39646), ('127.0.0.1', 43981)]

The second demonstration will show that we can take a Dask cuDF and utilize NCCL on the workers hosting its partitions. This includes:
1. Initializing a NCCL clique with the workers that host partitions underlying the Dask cuDF
2. Perform a collective comm operation across the partitions using only the cuML comms facade to communicate with the NCCL comm. 
3. Output a Dask cuDF with the results of the collective comm. 

In [30]:
X = gen_dask_cudf(500000, 50, demo)

result = demo.test_end_to_end(X)
print(str(result))

<dask_cudf.DataFrame | 2 tasks | 1 npartitions>


In [31]:
print(str(result.compute()))

     0    1    2    3    4    5    6 ...   49
0  2.0  2.0  2.0  2.0  2.0  2.0  2.0 ...  2.0
1  2.0  2.0  2.0  2.0  2.0  2.0  2.0 ...  2.0
2  2.0  2.0  2.0  2.0  2.0  2.0  2.0 ...  2.0
3  2.0  2.0  2.0  2.0  2.0  2.0  2.0 ...  2.0
4  2.0  2.0  2.0  2.0  2.0  2.0  2.0 ...  2.0
5  2.0  2.0  2.0  2.0  2.0  2.0  2.0 ...  2.0
6  2.0  2.0  2.0  2.0  2.0  2.0  2.0 ...  2.0
7  2.0  2.0  2.0  2.0  2.0  2.0  2.0 ...  2.0
8  2.0  2.0  2.0  2.0  2.0  2.0  2.0 ...  2.0
9  2.0  2.0  2.0  2.0  2.0  2.0  2.0 ...  2.0
[499990 more rows]
[42 more columns]
